# Recommendations with IBM

In this notebook, you will be putting your recommendation skills to use on real data from the IBM Watson Studio platform. 


You may either submit your notebook through the workspace here, or you may work from your local machine and submit through the next page.  Either way assure that your code passes the project [RUBRIC](https://review.udacity.com/#!/rubrics/2322/view).  **Please save regularly.**

By following the table of contents, you will build out a number of different methods for making recommendations that can be used for different situations. 


## Table of Contents

I. [Exploratory Data Analysis](#Exploratory-Data-Analysis)<br>
II. [Rank Based Recommendations](#Rank)<br>
III. [User-User Based Collaborative Filtering](#User-User)<br>
IV. [Content Based Recommendations (EXTRA - NOT REQUIRED)](#Content-Recs)<br>
V. [Matrix Factorization](#Matrix-Fact)<br>
VI. [Extras & Concluding](#conclusions)

At the end of the notebook, you will find directions for how to submit your work.  Let's get started by importing the necessary libraries and reading in the data.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import project_tests as t
import pickle

%matplotlib inline

df = pd.read_csv('data/user-item-interactions.csv')
df_content = pd.read_csv('data/articles_community.csv')
del df['Unnamed: 0']
del df_content['Unnamed: 0']

In [2]:
# Show df to get an idea of the data
df.head()

,article_id,title,email
0,1430.0,"using pixiedust for fast, flexible, and easier...",ef5f11f77ba020cd36e1105a00ab868bbdbf7fe7
1,1314.0,healthcare python streaming application demo,083cbdfa93c8444beaa4c5f5e0f5f9198e4f9e0b
2,1429.0,use deep learning for image classification,b96a4f2e92d8572034b1e9b28f9ac673765cd074
3,1338.0,ml optimization using cognitive assistant,06485706b34a5c9bf2a0ecdac41daf7e7654ceb7
4,1276.0,deploy your python model as a restful api,f01220c46fc92c6e6b161b1849de11faacd7ccb2


In [3]:
# Show df_content to get an idea of the data
df_content.head()

,doc_body,doc_description,doc_full_name,doc_status,article_id
0,Skip navigation Sign in SearchLoading...\r\n\r...,Detect bad readings in real time using Python ...,Detect Malfunctioning IoT Sensors with Streami...,Live,0
1,No Free Hunch Navigation * kaggle.com\r\n\r\n ...,"See the forest, see the trees. Here lies the c...",Communicating data science: A guide to present...,Live,1
2,☰ * Login\r\n * Sign Up\r\n\r\n * Learning Pat...,Here’s this week’s news in Data Science and Bi...,"This Week in Data Science (April 18, 2017)",Live,2
3,"DATALAYER: HIGH THROUGHPUT, LOW LATENCY AT SCA...",Learn how distributed DBs solve the problem of...,DataLayer Conference: Boost the performance of...,Live,3
4,Skip navigation Sign in SearchLoading...\r\n\r...,This video demonstrates the power of IBM DataS...,Analyze NY Restaurant data using Spark in DSX,Live,4


In [4]:
df.shape, df_content.shape

((45993, 3), (1056, 5))

### <a class="anchor" id="Exploratory-Data-Analysis">Part I : Exploratory Data Analysis</a>

Use the dictionary and cells below to provide some insight into the descriptive statistics of the data.

`1.` What is the distribution of how many articles a user interacts with in the dataset?  Provide a visual and descriptive statistics to assist with giving a look at the number of times each user interacts with an article.  

In [5]:
## Question 1:
num_users = len(df.groupby('email')['article_id'].count().sort_values())
print('There are totally %d of user IDs.' %num_users)

print('Now, take the %d-th of the reader on the rank of number of reading. This user read %d articles.' 
%(int(num_users/2), df.groupby('email')['article_id'].count().sort_values()[int(num_users/2)-1]) )

There are totally 5148 of user IDs.
Now, take the 2574-th of the reader on the rank of number of reading. This user read 3 articles.


In [6]:
#df = df.drop_duplicates()
#df.shape

In [7]:
## Question 2: take the number of reading for the top 1 reader
df['email'].value_counts()[:3]

2b6c0f514c2f2b04ad3c4583407dccd0810469ee    364
77959baaa9895a7e2bdc9297f8b27c1b6f2cb52a    363
2f5c7feae533ce046f2cb16fb3a29fe00528ed66    170
Name: email, dtype: int64

In [8]:
# Fill in the median and maximum number of user_article interactios below

median_val = 3 # 50% of individuals interact with ____ number of articles or fewer.
max_views_by_user = 364 # The maximum number of user-article interactions by any 1 user is ______.

`2.` Explore and remove duplicate articles from the **df_content** dataframe.  

In [9]:
# Find and explore duplicate articles

In [10]:
# Remove any rows that have the same article_id - only keep the first
df_content = df_content.drop_duplicates(subset=['article_id'])
df_content.shape

(1051, 5)

In [11]:
df_content['doc_status'].value_counts()

Live    1051
Name: doc_status, dtype: int64

In [12]:
len(df_content['doc_body'][1]), df_content['doc_description'][1], df_content['doc_full_name'][1]

(21271,
 'See the forest, see the trees. Here lies the challenge in both performing and presenting an analysis. As data scientists, analysts, and machine learning engineers faced with fulfilling business obj…',
 'Communicating data science: A guide to presenting your work')

`3.` Use the cells below to find:

**a.** The number of unique articles that have an interaction with a user.  
**b.** The number of unique articles in the dataset (whether they have any interactions or not).<br>
**c.** The number of unique users in the dataset. (excluding null values) <br>
**d.** The number of user-article interactions in the dataset.

In [13]:
## Question 1:
print(df['article_id'].nunique())

## Question 2:
print(df_content['article_id'].nunique())

## Question 3:
print(df['email'].nunique())

## Question 4:
#If a user interacts with an article 3 times, these are considered 3 separate interactions.
print(df.shape[0])

714
1051
5148
45993


In [14]:
unique_articles = 714 # The number of unique articles that have at least one interaction
total_articles = 1051 # The number of unique articles on the IBM platform
unique_users = 5148 # The number of unique users
user_article_interactions = 45993 # The number of user-article interactions

`4.` Use the cells below to find the most viewed **article_id**, as well as how often it was viewed.  After talking to the company leaders, the `email_mapper` function was deemed a reasonable way to map users to ids.  There were a small number of null values, and it was found that all of these null values likely belonged to a single user (which is how they are stored using the function below).

In [15]:
## Question 1 & 2:
df['article_id'].value_counts()

1429.0    937
1330.0    927
1431.0    671
1427.0    643
1364.0    627
1314.0    614
1293.0    572
1170.0    565
1162.0    512
1304.0    483
1436.0    481
1271.0    473
1398.0    465
43.0      460
1351.0    457
1393.0    455
1185.0    442
1160.0    433
1354.0    426
1368.0    418
1305.0    413
1338.0    382
1336.0    379
1165.0    372
1172.0    363
151.0     352
1276.0    347
1432.0    340
1430.0    336
1052.0    330
         ... 
1208.0      2
778.0       2
1283.0      2
1135.0      2
1210.0      2
430.0       2
416.0       2
1251.0      2
1227.0      2
1230.0      2
1086.0      2
1156.0      2
1266.0      1
653.0       1
1233.0      1
1127.0      1
1092.0      1
1119.0      1
675.0       1
724.0       1
1237.0      1
1202.0      1
1344.0      1
662.0       1
1200.0      1
974.0       1
417.0       1
1113.0      1
984.0       1
1072.0      1
Name: article_id, Length: 714, dtype: int64

In [16]:
most_viewed_article_id = "1429.0" # The most viewed article in the dataset as a string with one value following the decimal 
max_views = 937 # The most viewed article in the dataset was viewed how many times?

In [17]:
## No need to change the code here - this will be helpful for later parts of the notebook
# Run this cell to map the user email to a user_id column and remove the email column

def email_mapper():
    coded_dict = dict()
    cter = 1
    email_encoded = []
    
    for val in df['email']:
        if val not in coded_dict:
            coded_dict[val] = cter
            cter+=1
        
        email_encoded.append(coded_dict[val])
    return email_encoded

email_encoded = email_mapper()
del df['email']
df['user_id'] = email_encoded

# show header
df.head()

,article_id,title,user_id
0,1430.0,"using pixiedust for fast, flexible, and easier...",1
1,1314.0,healthcare python streaming application demo,2
2,1429.0,use deep learning for image classification,3
3,1338.0,ml optimization using cognitive assistant,4
4,1276.0,deploy your python model as a restful api,5


In [18]:
# Notice: unique_users = 5148 # The number of unique users #Can ignore NaN values.
# If you count the None here, it becomes 5149
print(df.shape)
print(df['user_id'].nunique()) 

(45993, 3)
5149


In [19]:
## If you stored all your results in the variable names above, 
## you shouldn't need to change anything in this cell

sol_1_dict = {
    '`50% of individuals have _____ or fewer interactions.`': median_val,
    '`The total number of user-article interactions in the dataset is ______.`': user_article_interactions,
    '`The maximum number of user-article interactions by any 1 user is ______.`': max_views_by_user,
    '`The most viewed article in the dataset was viewed _____ times.`': max_views,
    '`The article_id of the most viewed article is ______.`': most_viewed_article_id,
    '`The number of unique articles that have at least 1 rating ______.`': unique_articles,
    '`The number of unique users in the dataset is ______`': unique_users,
    '`The number of unique articles on the IBM platform`': total_articles
}

# Test your dictionary against the solution
t.sol_1_test(sol_1_dict)

It looks like you have everything right here! Nice job!


### <a class="anchor" id="Rank">Part II: Rank-Based Recommendations</a>

Unlike in the earlier lessons, we don't actually have ratings for whether a user liked an article or not.  We only know that a user has interacted with an article.  In these cases, the popularity of an article can really only be based on how often an article was interacted with.

`1.` Fill in the function below to return the **n** top articles ordered with most interactions as the top. Test your function using the tests below.

In [20]:
df_count = df.groupby('title')['user_id'].count() \
            .reset_index(name='count') \
            .sort_values('count', ascending=False)
list(df_count[:5]['title'])

['use deep learning for image classification',
 'insights from new york car accident reports',
 'visualize car data with brunel',
 'use xgboost, scikit-learn & ibm watson machine learning apis',
 'predicting churn with the spss random tree algorithm']

In [21]:
df.query('article_id == 1429.0')['title'].iloc[0]

'use deep learning for image classification'

In [22]:
def get_top_articles(n, df=df):
    '''
    INPUT:
        n - (int) the number of top articles to return
        df - (pandas dataframe) df as defined at the top of the notebook 
    
    OUTPUT:
        top_articles - (list) A list of the top 'n' article titles 
    
    '''
    top_articles = df.groupby('title')['user_id'].count() \
                    .reset_index(name='count') \
                    .sort_values('count', ascending=False)['title'][:n]
    
    return top_articles # Return the top article titles from df (not df_content)

def get_top_article_ids(n, df=df):
    '''
    INPUT:
        n - (int) the number of top articles to return
        df - (pandas dataframe) df as defined at the top of the notebook 
    
    OUTPUT:
        top_articles - (list) A list of the top 'n' article ids 
    
    '''
    top_articles = list(df.groupby('article_id')['user_id'].count() \
                    .reset_index(name='count') \
                    .sort_values('count', ascending=False)['article_id'][:n])
 
    return top_articles # Return the top article ids

In [23]:
print(get_top_articles(10))
print(get_top_article_ids(10))

637           use deep learning for image classification
396          insights from new york car accident reports
668                       visualize car data with brunel
650    use xgboost, scikit-learn & ibm watson machine...
485    predicting churn with the spss random tree alg...
356         healthcare python streaming application demo
323    finding optimal locations of new store using d...
151             apache spark lab, part 1: basic concepts
133              analyze energy consumption in buildings
345    gosales transactions for logistic regression m...
Name: title, dtype: object
[1429.0, 1330.0, 1431.0, 1427.0, 1364.0, 1314.0, 1293.0, 1170.0, 1162.0, 1304.0]


In [24]:
# Test your function by returning the top 5, 10, and 20 articles
top_5 = get_top_articles(5)
top_10 = get_top_articles(10)
top_20 = get_top_articles(20)

# Test each of your three lists from above
t.sol_2_test(get_top_articles)

Your top_5 looks like the solution list! Nice job.
Your top_10 looks like the solution list! Nice job.
Your top_20 looks like the solution list! Nice job.


### <a class="anchor" id="User-User">Part III: User-User Based Collaborative Filtering</a>


`1.` Use the function below to reformat the **df** dataframe to be shaped with users as the rows and articles as the columns.  

* Each **user** should only appear in each **row** once.


* Each **article** should only show up in one **column**.  


* **If a user has interacted with an article, then place a 1 where the user-row meets for that article-column**.  It does not matter how many times a user has interacted with the article, all entries where a user has interacted with an article should be a 1.  


* **If a user has not interacted with an item, then place a zero where the user-row meets for that article-column**. 

Use the tests to make sure the basic structure of your matrix matches what is expected by the solution.

In [25]:
# create the user-article matrix with 1's and 0's

def create_user_item_matrix(df):
    '''
    INPUT:
        df - pandas dataframe with article_id, title, user_id columns
    
    OUTPUT:
        user_item - user item matrix 
    
    Description:
    Return a matrix with user ids as rows and article ids on the columns with 1 values where a user interacted with 
    an article and a 0 otherwise
    '''
    user_item = df.drop_duplicates().groupby(['user_id', 'article_id']).size().unstack()
    user_item = user_item.fillna(0)
    
    return user_item # return the user_item matrix 

user_item = create_user_item_matrix(df)

In [26]:
## Tests: You should just need to run this cell.  Don't change the code.
assert user_item.shape[0] == 5149, "Oops!  The number of users in the user-article matrix doesn't look right."
assert user_item.shape[1] == 714, "Oops!  The number of articles in the user-article matrix doesn't look right."
assert user_item.sum(axis=1)[1] == 36, "Oops!  The number of articles seen by user 1 doesn't look right."
print("You have passed our quick tests!  Please proceed!")

You have passed our quick tests!  Please proceed!


`2.` Complete the function below which should take a user_id and provide an ordered list of the most similar users to that user (from most similar to least similar).  The returned result should not contain the provided user_id, as we know that each user is similar to him/herself. Because the results for each user here are binary, it (perhaps) makes sense to compute similarity as the dot product of two users. 

Use the tests to test your function.

In [27]:
user_item.head()

article_id,0.0,2.0,4.0,8.0,9.0,12.0,14.0,15.0,16.0,18.0,...,1434.0,1435.0,1436.0,1437.0,1439.0,1440.0,1441.0,1442.0,1443.0,1444.0
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
user_item.index[:30]

Int64Index([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
            18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30],
           dtype='int64', name='user_id')

In [29]:
for u in user_item.index[:5]:
    #print(user_item.iloc[u-1])
    dist = np.sqrt(np.sum((np.array(user_item.iloc[u-1])- np.array(user_item.iloc[0]))**2))
    user = u
    print(dist, user)

0.0 1
6.16441400297 2
8.0 3
7.48331477355 4
6.2449979984 5


In [31]:
np.array(user_item).shape

(5149, 714)

In [32]:
test = list(np.dot(np.array(user_item[user_item.index==1.0]), np.array(user_item).T)[0])
test.sort(reverse=True)
test

[36.0,
 35.0,
 17.0,
 17.0,
 15.0,
 15.0,
 14.0,
 14.0,
 13.0,
 13.0,
 12.0,
 12.0,
 12.0,
 12.0,
 11.0,
 11.0,
 11.0,
 11.0,
 10.0,
 10.0,
 10.0,
 10.0,
 10.0,
 10.0,
 10.0,
 10.0,
 9.0,
 9.0,
 9.0,
 9.0,
 9.0,
 9.0,
 9.0,
 9.0,
 9.0,
 9.0,
 9.0,
 9.0,
 9.0,
 9.0,
 9.0,
 9.0,
 9.0,
 9.0,
 9.0,
 9.0,
 9.0,
 8.0,
 8.0,
 8.0,
 8.0,
 8.0,
 8.0,
 8.0,
 8.0,
 8.0,
 8.0,
 8.0,
 8.0,
 8.0,
 8.0,
 8.0,
 8.0,
 8.0,
 8.0,
 8.0,
 8.0,
 8.0,
 7.0,
 7.0,
 7.0,
 7.0,
 7.0,
 7.0,
 7.0,
 7.0,
 7.0,
 7.0,
 7.0,
 7.0,
 7.0,
 7.0,
 7.0,
 7.0,
 7.0,
 7.0,
 7.0,
 7.0,
 7.0,
 7.0,
 7.0,
 7.0,
 7.0,
 7.0,
 7.0,
 7.0,
 6.0,
 6.0,
 6.0,
 6.0,
 6.0,
 6.0,
 6.0,
 6.0,
 6.0,
 6.0,
 6.0,
 6.0,
 6.0,
 6.0,
 6.0,
 6.0,
 6.0,
 6.0,
 6.0,
 6.0,
 6.0,
 6.0,
 6.0,
 6.0,
 6.0,
 6.0,
 6.0,
 6.0,
 6.0,
 6.0,
 6.0,
 6.0,
 6.0,
 6.0,
 6.0,
 6.0,
 6.0,
 6.0,
 6.0,
 6.0,
 6.0,
 6.0,
 6.0,
 6.0,
 6.0,
 6.0,
 6.0,
 6.0,
 6.0,
 6.0,
 6.0,
 6.0,
 6.0,
 6.0,
 6.0,
 6.0,
 6.0,
 6.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5

In [33]:
test = pd.Series(np.dot(np.array(user_item[user_item.index==1.0]), np.array(user_item).T)[0], index=user_item.index)
test.sort_values(ascending=False)[1:].index.tolist()

[3933,
 23,
 3782,
 203,
 4459,
 131,
 3870,
 46,
 4201,
 5041,
 49,
 3697,
 395,
 3910,
 322,
 3622,
 242,
 4642,
 290,
 2982,
 912,
 3540,
 98,
 754,
 3764,
 256,
 52,
 268,
 40,
 371,
 3596,
 621,
 21,
 273,
 3637,
 5138,
 135,
 204,
 3784,
 3775,
 1355,
 4785,
 4932,
 4134,
 3621,
 3651,
 3483,
 696,
 4293,
 488,
 4892,
 3532,
 334,
 4038,
 184,
 1897,
 619,
 765,
 2926,
 214,
 249,
 4778,
 186,
 3141,
 4755,
 3136,
 64,
 3740,
 54,
 4934,
 3684,
 125,
 72,
 3856,
 4484,
 445,
 58,
 3632,
 60,
 67,
 2790,
 5013,
 4206,
 3024,
 4209,
 665,
 5079,
 656,
 5140,
 4706,
 4774,
 288,
 187,
 10,
 38,
 4471,
 4824,
 2975,
 223,
 2981,
 3693,
 90,
 193,
 807,
 3006,
 3169,
 195,
 409,
 211,
 87,
 235,
 330,
 3197,
 244,
 324,
 4595,
 4933,
 2430,
 3408,
 295,
 4883,
 4543,
 3572,
 3578,
 3358,
 362,
 3354,
 113,
 3353,
 111,
 1040,
 263,
 261,
 1059,
 3485,
 170,
 379,
 4277,
 689,
 733,
 591,
 3818,
 4404,
 69,
 2161,
 4225,
 511,
 3949,
 3,
 28,
 3879,
 3794,
 8,
 2903,
 3305,
 155,
 4021

In [34]:
def find_similar_users(user_id, user_item=user_item):
    '''
    INPUT:
        user_id - (int) a user_id
        user_item - (pandas dataframe) matrix of users by articles: 
                1's when a user has interacted with an article, 0 otherwise
    
    OUTPUT:
        similar_users - (list) an ordered list where the closest users (largest dot product users)
                    are listed first
    
    Description:
    Computes the similarity of every pair of users based on the dot product
    Returns an ordered
    
    '''
    # compute similarity of each user to the provided user
    dist = dict()
    for u in user_item.index:
        dist[u] = np.linalg.norm( np.array(user_item.iloc[u-1] - user_item.iloc[user_id-1]) )
     
    # sort by similarity
    sorted_dist = dict( sorted(dist.items(), key=lambda x: x[1]))
    #print(sorted_dist)
    
    # create list of just the ids
    similar_users = list(sorted_dist.keys())
   
    # remove the own user's id
    similar_users = similar_users[1:]
        
    return similar_users # return a list of the users in order from most to least similar

In [35]:
find_similar_users(1) 
# If you print out the sorted_dist, you will found a bunch of the ids holding the same similarity 
# ex: find "5.8309518948453007", for this kind of cases, they're ordered by the ids

[3933,
 253,
 824,
 5034,
 136,
 2305,
 2447,
 3031,
 4020,
 4504,
 254,
 325,
 329,
 337,
 650,
 863,
 884,
 890,
 1154,
 1229,
 1274,
 1413,
 1427,
 1513,
 1620,
 1752,
 1775,
 1780,
 1988,
 2128,
 2174,
 2406,
 2458,
 2485,
 2521,
 2586,
 2667,
 2710,
 2725,
 2788,
 2800,
 2819,
 2897,
 3322,
 3435,
 3913,
 4444,
 4510,
 4638,
 4716,
 4749,
 99,
 146,
 189,
 201,
 247,
 264,
 282,
 302,
 327,
 331,
 338,
 340,
 348,
 353,
 374,
 392,
 394,
 396,
 397,
 400,
 428,
 436,
 452,
 465,
 475,
 504,
 505,
 524,
 544,
 566,
 594,
 605,
 610,
 611,
 615,
 625,
 631,
 793,
 849,
 894,
 904,
 905,
 921,
 930,
 931,
 1001,
 1103,
 1104,
 1113,
 1153,
 1196,
 1205,
 1220,
 1231,
 1257,
 1276,
 1294,
 1319,
 1320,
 1358,
 1359,
 1377,
 1398,
 1417,
 1431,
 1438,
 1462,
 1512,
 1518,
 1521,
 1553,
 1578,
 1595,
 1597,
 1643,
 1647,
 1650,
 1658,
 1661,
 1665,
 1726,
 1756,
 1766,
 1778,
 1785,
 1787,
 1789,
 1838,
 1852,
 1880,
 1885,
 1896,
 1898,
 1910,
 1938,
 1950,
 1965,
 1986,
 2000,
 2005,


In [36]:
# Do a spot check of your function
print("The 10 most similar users to user 1 are: {}".format(find_similar_users(1)[:10]))
print("The 5 most similar users to user 3933 are: {}".format(find_similar_users(3933)[:5]))
print("The 3 most similar users to user 46 are: {}".format(find_similar_users(46)[:3]))

The 10 most similar users to user 1 are: [3933, 253, 824, 5034, 136, 2305, 2447, 3031, 4020, 4504]
The 5 most similar users to user 3933 are: [1, 253, 824, 5034, 136]
The 3 most similar users to user 46 are: [4201, 790, 248]


`3.` Now that you have a function that provides the most similar users to each user, you will want to use these users to find articles you can recommend.  Complete the functions below to return the articles you would recommend to each user. 

In [37]:
def get_article_names(article_ids, df=df):
    '''
    INPUT:
        article_ids - (list) a list of article ids
        df - (pandas dataframe) df as defined at the top of the notebook
    
    OUTPUT:
        article_names - (list) a list of article names associated with the list of article ids 
                    (this is identified by the title column)
    '''
    article_names = []
    for i in article_ids:
        article_names.append(df[df['article_id'] == float(i)].iloc[0].title)
    
    return article_names # Return the article names associated with list of article ids


def get_user_articles(user_id, user_item=user_item):
    '''
    INPUT:
        user_id - (int) a user id
        user_item - (pandas dataframe) matrix of users by articles: 
                1's when a user has interacted with an article, 0 otherwise
    
    OUTPUT:
        article_ids - (list) a list of the article ids seen by the user
        article_names - (list) a list of article names associated with the list of article ids 
                    (this is identified by the doc_full_name column in df_content)
    
    Description:
    Provides a list of the article_ids and article titles that have been seen by a user
    '''
    article_ids, article_names = [], []
    
    for count, i in enumerate(user_item.iloc[user_id-1]):
        if i != 0.:
            article_ids.append(str(user_item.columns[count]))
            
    article_names = get_article_names(article_ids, df)
    
    return article_ids, article_names # return the ids and names


def user_user_recs(user_id, m=10):
    '''
    INPUT:
        user_id - (int) a user id
        m - (int) the number of recommendations you want for the user
    
    OUTPUT:
        recs - (list) a list of recommendations for the user
    
    Description:
    Loops through the users based on closeness to the input user_id
    For each user - finds articles the user hasn't seen before and provides them as recs
    Does this until m recommendations are found
    
    Notes:
    Users who are the same closeness are chosen arbitrarily as the 'next' user
    
    For the user where the number of recommended articles starts below m 
    and ends exceeding m, the last items are chosen arbitrarily
    
    '''
    
    # Get article_ids and article titles that have been seen by a given user
    seen_article_ids, seen_article_names = get_user_articles(user_id, user_item=user_item)
    
    recs = []
    
    # Loop over similar users
    for u in find_similar_users(user_id):
        # Get article_ids and article titles that have been seen by the similar user
        article_ids, article_names = get_user_articles(u, user_item=user_item)
        
        # Remove the articles from the similar user that have been seen by the given user
        similar_article_ids = list(np.setdiff1d(article_ids, seen_article_ids))
        
        # If needs more recommendations
        if(len(recs) < m):
            recs.extend(similar_article_ids)
            recs = list(set(recs))
            
        # If recommendations are filled more than needed, remove the last few (arbitrarily)   
        if(len(recs) > m):
            recs = recs[:10]
            break
    
    return recs # return your recommendations for this user_id    

In [38]:
# Check Results
get_article_names(user_user_recs(1, 10)) # Return 10 recommendations for user 1

['self-service data preparation with ibm data refinery',
 'this week in data science (may 30, 2017)',
 '9 mistakes to avoid when starting your career in data science',
 'movie recommender system with spark machine learning',
 'analyze precipitation data',
 'ml optimization using cognitive assistant',
 'getting started with python',
 'use spark for python to load data and run sql queries',
 'the nurse assignment problem',
 'the machine learning database']

In [39]:
# Test your functions here - No need to change this code - just run this cell
assert set(get_article_names(['1024.0', '1176.0', '1305.0', '1314.0', '1422.0', '1427.0'])) == set(['using deep learning to reconstruct high-resolution audio', 'build a python app on the streaming analytics service', 'gosales transactions for naive bayes model', 'healthcare python streaming application demo', 'use r dataframes & ibm watson natural language understanding', 'use xgboost, scikit-learn & ibm watson machine learning apis']), "Oops! Your the get_article_names function doesn't work quite how we expect."
assert set(get_article_names(['1320.0', '232.0', '844.0'])) == set(['housing (2015): united states demographic measures','self-service data preparation with ibm data refinery','use the cloudant-spark connector in python notebook']), "Oops! Your the get_article_names function doesn't work quite how we expect."
assert set(get_user_articles(20)[0]) == set(['1320.0', '232.0', '844.0'])
assert set(get_user_articles(20)[1]) == set(['housing (2015): united states demographic measures', 'self-service data preparation with ibm data refinery','use the cloudant-spark connector in python notebook'])
assert set(get_user_articles(2)[0]) == set(['1024.0', '1176.0', '1305.0', '1314.0', '1422.0', '1427.0'])
assert set(get_user_articles(2)[1]) == set(['using deep learning to reconstruct high-resolution audio', 'build a python app on the streaming analytics service', 'gosales transactions for naive bayes model', 'healthcare python streaming application demo', 'use r dataframes & ibm watson natural language understanding', 'use xgboost, scikit-learn & ibm watson machine learning apis'])
print("If this is all you see, you passed all of our tests!  Nice job!")

If this is all you see, you passed all of our tests!  Nice job!


`4.` Now we are going to improve the consistency of the **user_user_recs** function from above.  

* Instead of arbitrarily choosing when we obtain users who are all the same closeness to a given user - choose the users that have the most total article interactions before choosing those with fewer article interactions.


* Instead of arbitrarily choosing articles from the user where the number of recommended articles starts below m and ends exceeding m, choose articles with the articles with the most total interactions before choosing those with fewer total interactions. This ranking should be  what would be obtained from the **top_articles** function you wrote earlier.

In [40]:
popularity = pd.DataFrame(df.groupby(['article_id']).size().to_frame('count').reset_index())
popularity.sort_values(['article_id','count'], ascending=[True, False])

,article_id,count
0,0.0,14
1,2.0,58
2,4.0,13
3,8.0,85
4,9.0,10
5,12.0,157
6,14.0,89
7,15.0,26
8,16.0,61
9,18.0,78


In [41]:
neighbors_df = pd.DataFrame()

neighbor_ids = find_similar_users(1)
neighbors_df['neighbor_id'] = neighbor_ids


similarities = []
num_interactions = []

for u in neighbor_ids:
    similarities.append(np.linalg.norm( np.array(user_item.iloc[u-1] - user_item.iloc[0]) ))
    num_interactions.append(user_item.iloc[u-1].sum())
    
neighbors_df['similarity'] = similarities
neighbors_df['num_interactions'] = num_interactions 
neighbors_df

,neighbor_id,similarity,num_interactions
0,3933,1.000000,35.0
1,253,5.656854,4.0
2,824,5.656854,4.0
3,5034,5.656854,4.0
4,136,5.744563,5.0
5,2305,5.744563,5.0
6,2447,5.744563,3.0
7,3031,5.744563,3.0
8,4020,5.744563,3.0
9,4504,5.744563,3.0


In [42]:
def get_top_sorted_users(user_id, df=df, user_item=user_item):
    '''
    INPUT:
        user_id - (int)
        df - (pandas dataframe) df as defined at the top of the notebook 
        user_item - (pandas dataframe) matrix of users by articles: 
            1's when a user has interacted with an article, 0 otherwise
    
            
    OUTPUT:
        neighbors_df - (pandas dataframe) a dataframe with:
                    neighbor_id - is a neighbor user_id
                    similarity - measure of the similarity of each user to the provided user_id
                    num_interactions - the number of articles viewed by the user - if a u
                    
    Other Details - sort the neighbors_df by the similarity and then by number of interactions where 
                    highest of each is higher in the dataframe
     
    '''
    neighbors_df = pd.DataFrame()

    similarities = []
    num_interactions = []

    # First use the funtion defined before to the similar users
    neighbor_ids = find_similar_users(user_id)
    neighbors_df['neighbor_id'] = neighbor_ids

    # Loop over the similar users and dump the dist and the number of articles viewed
    for u in neighbor_ids:
        similarities.append(np.linalg.norm( np.array(user_item.iloc[u-1] - user_item.iloc[0]) ))
        num_interactions.append(user_item.iloc[u-1].sum())
    
    neighbors_df['similarity'] = similarities
    neighbors_df['num_interactions'] = num_interactions 
    
    # It's been sorted by the dist, add sorting on the number of articles viewed
    neighbors_df.sort_values(['similarity', 'num_interactions'], ascending=[True, False], inplace=True)
    
    return neighbors_df # Return the dataframe specified in the doc_string


def user_user_recs_part2(user_id, m=10):
    '''
    INPUT:
        user_id - (int) a user id
        m - (int) the number of recommendations you want for the user
    
    OUTPUT:
        recs - (list) a list of recommendations for the user by article id
        rec_names - (list) a list of recommendations for the user by article title
    
    Description:
    Loops through the users based on closeness to the input user_id
    For each user - finds articles the user hasn't seen before and provides them as recs
    Does this until m recommendations are found
    
    Notes:
    * Choose the users that have the most total article interactions 
    before choosing those with fewer article interactions.

    * Choose articles with the most total interactions 
    before choosing those with fewer total interactions. 
   
    '''    
    # Get similar user ids
    similar_users = get_top_sorted_users(user_id, df=df, user_item=user_item)['neighbor_id']
    
    # Get article_ids and article titles that have been seen by a given user
    seen_article_ids, seen_article_names = get_user_articles(user_id, user_item=user_item)
    
    # Create a reference dataframe with columns of article_id and its number of viewed
    popularity = pd.DataFrame(df.groupby(['article_id']).size().to_frame('count').reset_index())
    popularity.sort_values(['article_id','count'], ascending=[True, False])
    
    recs, rec_names = [], []
    
    # Loop over similar users
    for u in similar_users:
        # Get article_ids and article titles that have been seen by the similar user
        article_ids, article_names = get_user_articles(u, user_item=user_item)
        
        # Remove the articles from the similar user that have been seen by the given user
        similar_article_ids = list(np.setdiff1d(article_ids, seen_article_ids))
        
        # Loop over the non-repeated recommendation articles from the similar user, 
        # and order it to the number of viewed. Divided into 2 steps:
        # Step 1: obtained the number of viewed
        counts = []
        for i in [float(i) for i in similar_article_ids]:
            counts.append(popularity[popularity['article_id'] == i]['count'].values)
            
        # Step 2: order 
        pair_list = list(zip(counts, similar_article_ids))
        pair_list.sort(reverse=True)
        similar_article_ids = [a for c,a in pair_list]
        
        
        # If needs more recommendations
        if(len(recs) < m):
            recs.extend(similar_article_ids)
            recs = list(set(recs))
            
        # If recommendations are filled more than needed, remove the last few (sorted already)   
        if(len(recs) > m):
            recs = recs[:10]
            break

    rec_names = get_article_names(recs, df=df)
    
    return recs, rec_names

In [43]:
popularity[popularity['article_id'] == 109.0]['count'].values 

array([198])

In [44]:
# Quick spot check - don't change this code - just use it to test your functions
rec_ids, rec_names = user_user_recs_part2(20, 10)
print("The top 10 recommendations for user 20 are the following article ids:")
print(rec_ids)
print()
print("The top 10 recommendations for user 20 are the following article names:")
print(rec_names)

The top 10 recommendations for user 20 are the following article ids:
['1305.0', '268.0', '494.0', '1427.0', '1400.0', '43.0', '1293.0', '981.0', '310.0', '1436.0']

The top 10 recommendations for user 20 are the following article names:
['gosales transactions for naive bayes model', 'sector correlations shiny app', 'python for loops explained (python for data science basics #5)', 'use xgboost, scikit-learn & ibm watson machine learning apis', 'uci ml repository: chronic kidney disease data set', 'deep learning with tensorflow course by big data university', 'finding optimal locations of new store using decision optimization', 'super fast string matching in python', 'time series prediction using recurrent neural networks (lstms)', 'welcome to pixiedust']


`5.` Use your functions from above to correctly fill in the solutions to the dictionary below.  Then test your dictionary against the solution.  Provide the code you need to answer each following the comments below.

In [45]:
### Tests with a dictionary of results


# Find the user that is most similar to user 1 
user1_most_sim = get_top_sorted_users(1, df=df, user_item=user_item)['neighbor_id'][0]

# Find the 10th most similar user to user 131
user131_10th_sim = get_top_sorted_users(131, df=df, user_item=user_item)['neighbor_id'][:10]

In [46]:
get_top_sorted_users(131, df=df, user_item=user_item)

,neighbor_id,similarity,num_interactions
4689,1,0.000000,36.0
4849,3933,1.000000,35.0
20,253,5.656854,4.0
27,824,5.656854,4.0
45,5034,5.656854,4.0
51,136,5.744563,5.0
3794,2305,5.744563,5.0
78,3031,5.744563,3.0
616,2447,5.744563,3.0
869,4504,5.744563,3.0


In [47]:
## Dictionary Test Here
sol_5_dict = {
    'The user that is most similar to user 1.': user1_most_sim, 
    'The user that is the 10th most similar to user 131': user131_10th_sim,
}

t.sol_5_test(sol_5_dict)

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

`6.` If we were given a new user, which of the above functions would you be able to use to make recommendations?  Explain.  Can you think of a better way we might make recommendations?  Use the cell below to explain a better method for new users.

**Provide your response here.**

`7.` Using your existing functions, provide the top 10 recommended articles you would provide for the a new user below.  You can test your function against our thoughts to make sure we are all on the same page with how we might make a recommendation.

In [48]:
new_user = '0.0'

# What would your recommendations be for this new user '0.0'?  As a new user, they have no observed articles.
# Provide a list of the top 10 article ids you would give to 

new_user_recs = get_top_article_ids(10, df=df) # Your recommendations here


In [49]:
print(new_user_recs)

[1429.0, 1330.0, 1431.0, 1427.0, 1364.0, 1314.0, 1293.0, 1170.0, 1162.0, 1304.0]


In [50]:
assert set(new_user_recs) == set([1314.0, 1429.0, 1293.0, 1427.0, 1162.0, 1364.0, 1304.0, 1170.0, 1431.0, 1330.0]), "Oops!  It makes sense that in this case we would want to recommend the most popular articles, because we don't know anything about these users."

print("That's right!  Nice job!")

That's right!  Nice job!


### <a class="anchor" id="Content-Recs">Part IV: Content Based Recommendations (EXTRA - NOT REQUIRED)</a>

Another method we might use to make recommendations is to perform a ranking of the highest ranked articles associated with some term.  You might consider content to be the **doc_body**, **doc_description**, or **doc_full_name**.  There isn't one way to create a content based recommendation, especially considering that each of these columns hold content related information.  

`1.` Use the function body below to create a content based recommender.  Since there isn't one right answer for this recommendation tactic, no test functions are provided.  Feel free to change the function inputs if you decide you want to try a method that requires more input values.  The input values are currently set with one idea in mind that you may use to make content based recommendations.  One additional idea is that you might want to choose the most popular recommendations that meet your 'content criteria', but again, there is a lot of flexibility in how you might make these recommendations.

### This part is NOT REQUIRED to pass this project.  However, you may choose to take this on as an extra way to show off your skills.

In [ ]:
def make_content_recs():
    '''
    INPUT:
    
    OUTPUT:
    
    '''

`2.` Now that you have put together your content-based recommendation system, use the cell below to write a summary explaining how your content based recommender works.  Do you see any possible improvements that could be made to your function?  Is there anything novel about your content based recommender?

### This part is NOT REQUIRED to pass this project.  However, you may choose to take this on as an extra way to show off your skills.

**Write an explanation of your content based recommendation system here.**

`3.` Use your content-recommendation system to make recommendations for the below scenarios based on the comments.  Again no tests are provided here, because there isn't one right answer that could be used to find these content based recommendations.

### This part is NOT REQUIRED to pass this project.  However, you may choose to take this on as an extra way to show off your skills.

In [ ]:
# make recommendations for a brand new user


# make a recommendations for a user who only has interacted with article id '1427.0'



### <a class="anchor" id="Matrix-Fact">Part V: Matrix Factorization</a>

In this part of the notebook, you will build use matrix factorization to make article recommendations to the users on the IBM Watson Studio platform.

`1.` You should have already created a **user_item** matrix above in **question 1** of **Part III** above.  This first question here will just require that you run the cells to get things set up for the rest of **Part V** of the notebook. 

In [89]:
# Load the matrix here
user_item_matrix = pd.read_pickle('user_item_matrix.p')

In [90]:
# quick look at the matrix
user_item_matrix.head()

article_id,0.0,100.0,1000.0,1004.0,1006.0,1008.0,101.0,1014.0,1015.0,1016.0,...,977.0,98.0,981.0,984.0,985.0,986.0,990.0,993.0,996.0,997.0
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


`2.` In this situation, you can use Singular Value Decomposition from [numpy](https://docs.scipy.org/doc/numpy-1.14.0/reference/generated/numpy.linalg.svd.html) on the user-item matrix.  Use the cell to perform SVD, and explain why this is different than in the lesson.

In [ ]:
# Perform SVD on the User-Item Matrix Here

u, s, vt = # use the built in to get the three matrices

**Provide your response here.**

`3.` Now for the tricky part, how do we choose the number of latent features to use?  Running the below cell, you can see that as the number of latent features increases, we obtain a lower error rate on making predictions for the 1 and 0 values in the user-item matrix.  Run the cell below to get an idea of how the accuracy improves as we increase the number of latent features.

In [ ]:
num_latent_feats = np.arange(10,700+10,20)
sum_errs = []

for k in num_latent_feats:
    # restructure with k latent features
    s_new, u_new, vt_new = np.diag(s[:k]), u[:, :k], vt[:k, :]
    
    # take dot product
    user_item_est = np.around(np.dot(np.dot(u_new, s_new), vt_new))
    
    # compute error for each prediction to actual value
    diffs = np.subtract(user_item_matrix, user_item_est)
    
    # total errors and keep track of them
    err = np.sum(np.sum(np.abs(diffs)))
    sum_errs.append(err)
    
    
plt.plot(num_latent_feats, 1 - np.array(sum_errs)/df.shape[0]);
plt.xlabel('Number of Latent Features');
plt.ylabel('Accuracy');
plt.title('Accuracy vs. Number of Latent Features');

`4.` From the above, we can't really be sure how many features to use, because simply having a better way to predict the 1's and 0's of the matrix doesn't exactly give us an indication of if we are able to make good recommendations.  Instead, we might split our dataset into a training and test set of data, as shown in the cell below.  

Use the code from question 3 to understand the impact on accuracy of the training and test sets of data with different numbers of latent features. Using the split below: 

* How many users can we make predictions for in the test set?  
* How many users are we not able to make predictions for because of the cold start problem?
* How many articles can we make predictions for in the test set?  
* How many articles are we not able to make predictions for because of the cold start problem?

In [ ]:
df_train = df.head(40000)
df_test = df.tail(5993)

def create_test_and_train_user_item(df_train, df_test):
    '''
    INPUT:
    df_train - training dataframe
    df_test - test dataframe
    
    OUTPUT:
    user_item_train - a user-item matrix of the training dataframe 
                      (unique users for each row and unique articles for each column)
    user_item_test - a user-item matrix of the testing dataframe 
                    (unique users for each row and unique articles for each column)
    test_idx - all of the test user ids
    test_arts - all of the test article ids
    
    '''
    # Your code here
    
    return user_item_train, user_item_test, test_idx, test_arts

user_item_train, user_item_test, test_idx, test_arts = create_test_and_train_user_item(df_train, df_test)

In [ ]:
# Replace the values in the dictionary below
a = 662 
b = 574 
c = 20 
d = 0 


sol_4_dict = {
    'How many users can we make predictions for in the test set?': # letter here, 
    'How many users in the test set are we not able to make predictions for because of the cold start problem?': # letter here, 
    'How many movies can we make predictions for in the test set?': # letter here,
    'How many movies in the test set are we not able to make predictions for because of the cold start problem?': # letter here
}

t.sol_4_test(sol_4_dict)

`5.` Now use the **user_item_train** dataset from above to find U, S, and V transpose using SVD. Then find the subset of rows in the **user_item_test** dataset that you can predict using this matrix decomposition with different numbers of latent features to see how many features makes sense to keep based on the accuracy on the test data. This will require combining what was done in questions `2` - `4`.

Use the cells below to explore how well SVD works towards making predictions for recommendations on the test data.  

In [ ]:
# fit SVD on the user_item_train matrix
u_train, s_train, vt_train = # fit svd similar to above then use the cells below

In [ ]:
# Use these cells to see how well you can use the training 
# decomposition to predict on test data

`6.` Use the cell below to comment on the results you found in the previous question. Given the circumstances of your results, discuss what you might do to determine if the recommendations you make with any of the above recommendation systems are an improvement to how users currently find articles? 

**Your response here.**

<a id='conclusions'></a>
### Extras
Using your workbook, you could now save your recommendations for each user, develop a class to make new predictions and update your results, and make a flask app to deploy your results.  These tasks are beyond what is required for this project.  However, from what you learned in the lessons, you certainly capable of taking these tasks on to improve upon your work here!


## Conclusion

> Congratulations!  You have reached the end of the Recommendations with IBM project! 

> **Tip**: Once you are satisfied with your work here, check over your report to make sure that it is satisfies all the areas of the [rubric](https://review.udacity.com/#!/rubrics/2322/view). You should also probably remove all of the "Tips" like this one so that the presentation is as polished as possible.


## Directions to Submit

> Before you submit your project, you need to create a .html or .pdf version of this notebook in the workspace here. To do that, run the code cell below. If it worked correctly, you should get a return code of 0, and you should see the generated .html file in the workspace directory (click on the orange Jupyter icon in the upper left).

> Alternatively, you can download this report as .html via the **File** > **Download as** submenu, and then manually upload it into the workspace directory by clicking on the orange Jupyter icon in the upper left, then using the Upload button.

> Once you've done this, you can submit your project by clicking on the "Submit Project" button in the lower right here. This will create and submit a zip file with this .ipynb doc and the .html or .pdf version you created. Congratulations! 

In [ ]:
from subprocess import call
call(['python', '-m', 'nbconvert', 'Recommendations_with_IBM.ipynb'])